In [1]:
from IPython.display import Markdown, display

display(Markdown("""
# 🎙️ Interactive Lipsync & Blinking Animator 🖼️

Welcome! This Google Colab notebook provides a user-friendly interface to create talking head animations with realistic lipsync and natural eye blinking.

**How it works:**
1.  **Upload a still image** of a character or person.
2.  **Provide audio:**
    * Paste a YouTube URL (audio will be extracted).
    * OR upload your own audio file (e.g., .wav, .mp3).
3.  The powerful **SadTalker AI model** will then generate an animated video.
4.  **View and download** your animated character!

This tool leverages pre-trained models, so you don't need to train anything yourself. Let's get started!
"""))


# 🎙️ Interactive Lipsync & Blinking Animator 🖼️

Welcome! This Google Colab notebook provides a user-friendly interface to create talking head animations with realistic lipsync and natural eye blinking.

**How it works:**
1.  **Upload a still image** of a character or person.
2.  **Provide audio:**
    * Paste a YouTube URL (audio will be extracted).
    * OR upload your own audio file (e.g., .wav, .mp3).
3.  The powerful **SadTalker AI model** will then generate an animated video.
4.  **View and download** your animated character!

This tool leverages pre-trained models, so you don't need to train anything yourself. Let's get started!


In [2]:
import os
from IPython.display import HTML, clear_output

# Display a message while setting up
display(HTML("<h3 style='color: orange;'>Setting up the magic runway... This might take a few minutes. Please wait. ✨</h3>"))

# Clone SadTalker repository if not already present
if not os.path.exists('SadTalker'):
    print("Cloning SadTalker repository...")
    !git clone https://github.com/OpenTalker/SadTalker.git &> /dev/null
else:
    print("SadTalker repository already exists.")

%cd SadTalker

# Install necessary libraries
print("Installing dependencies...")
# Using specific versions known to work well in Colab, adjust if needed
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118 -q
!pip install -r requirements.txt -q
!pip install ipywidgets yt-dlp ffmpeg-python -q

# Download pre-trained models for SadTalker
print("Downloading SadTalker pre-trained models (if not already present)...")
if not os.path.exists('checkpoints'):
    !bash scripts/download_models.sh &> /dev/null
else:
    print("Checkpoints directory already exists. Assuming models are downloaded.")

%cd ..
clear_output(wait=True) # Clear the setup messages
display(HTML("<h3 style='color: green;'>✅ Setup Complete! You can now proceed to Step 2.</h3>"))

In [6]:
#@title ✨ Step 2: Upload Your Character & Audio, then Animate!

import ipywidgets as widgets
from ipywidgets import VBox, HBox, Layout, Tab, RadioButtons, Text, FileUpload, Button, Dropdown, Checkbox # Explicitly import all used widgets
from IPython.display import display, HTML, clear_output, Audio, FileLink
import yt_dlp
import os
import shutil
import glob
import time # For unique output filenames

# --- UI Element Definitions ---
style = {'description_width': 'initial'}
l_95width = Layout(width='95%')

# --- Output Areas ---
status_output = widgets.Output()
video_output_area = widgets.Output()

# --- Input Widgets ---
source_image_uploader = FileUpload( # Using FileUpload directly
    accept='image/*',
    multiple=False,
    description='1. Upload Character Image:',
    style=style,
    layout=l_95width
)
uploaded_image_display = widgets.Output(layout=Layout(max_width='300px', max_height='300px')) # For preview

audio_source_type = RadioButtons( # Using RadioButtons directly
    options=['YouTube URL', 'Upload Audio File'],
    value='YouTube URL',
    description='2. Audio Source:',
    style=style,
    disabled=False
)

youtube_url_widget = Text( # Using Text directly
    placeholder='Enter YouTube Video URL',
    description='YouTube URL:',
    style=style,
    layout=l_95width
)

audio_file_uploader = FileUpload( # Using FileUpload directly
    accept='audio/*',
    multiple=False,
    description='Upload Audio File:',
    style=style,
    layout=Layout(display='none') # Initially hidden
)
uploaded_audio_display = widgets.Output() # For preview/info

enhancer_widget = Dropdown( # Using Dropdown directly
    options=['gfpgan', 'RestoreFormer', 'None'],
    value='gfpgan',
    description='3. Face Enhancer:',
    style=style
)

still_mode_widget = Checkbox( # Using Checkbox directly
    value=True,
    description='Enable Still Mode (reduces head movement, good for portraits)',
    style=style,
    indent=False
)

generate_button = Button( # Using Button directly
    description='🎬 Generate Animation!',
    button_style='success',
    tooltip='Click to start generating the video',
    icon='magic',
    layout=Layout(width='auto', margin='20px 0 10px 0')
)

# --- Global paths for saved files ---
source_image_path_global = None
driving_audio_path_global = None

# --- Event Handlers ---
def on_audio_source_type_change(change):
    if change['new'] == 'YouTube URL':
        youtube_url_widget.layout.display = 'flex'
        audio_file_uploader.layout.display = 'none'
    else: # 'Upload Audio File'
        youtube_url_widget.layout.display = 'none'
        audio_file_uploader.layout.display = 'flex'

audio_source_type.observe(on_audio_source_type_change, names='value')
on_audio_source_type_change({'new': audio_source_type.value}) # Initialize

def handle_image_upload(change):
    global source_image_path_global
    with uploaded_image_display:
        clear_output(wait=True)
        if change['new']:
            uploader = change['owner']
            # In new ipywidgets, value is a tuple of dicts. Let's iterate.
            for file_info in uploader.value: # Iterate over the tuple of uploaded files
                filename = file_info['name']
                content = file_info['content']
                os.makedirs('inputs/source_image', exist_ok=True)
                source_image_path_global = f'inputs/source_image/{filename}'
                with open(source_image_path_global, 'wb') as f:
                    f.write(content)
                display(HTML(f"<b style='color:green;'>Uploaded:</b> {filename}<br><img src='{source_image_path_global}' style='max-width:250px; max-height:250px; margin-top:5px;'/>"))
                break # Process only the first file if multiple=False (though it should be one)
        else:
            source_image_path_global = None
            display(HTML("<p style='color:grey;'>No image uploaded.</p>"))

source_image_uploader.observe(handle_image_upload, names='value')

def handle_audio_upload(change):
    global driving_audio_path_global
    with uploaded_audio_display:
        clear_output(wait=True)
        if change['new']:
            uploader = change['owner']
            for file_info in uploader.value: # Iterate over the tuple of uploaded files
                filename = file_info['name']
                content = file_info['content']
                os.makedirs('inputs/driven_audio', exist_ok=True)
                driving_audio_path_global = f'inputs/driven_audio/{filename}'
                with open(driving_audio_path_global, 'wb') as f:
                    f.write(content)
                display(HTML(f"<b style='color:green;'>Uploaded:</b> {filename}"))
                display(Audio(driving_audio_path_global))
                break # Process only the first file
        else:
            driving_audio_path_global = None
            display(HTML("<p style='color:grey;'>No audio file uploaded.</p>"))

audio_file_uploader.observe(handle_audio_upload, names='value')


# --- Main Generate Function ---
def on_generate_button_clicked(b):
    global source_image_path_global, driving_audio_path_global
    with status_output:
        clear_output(wait=True)
        display(HTML("<h3 style='color: orange;'>⏳ Generating animation... Please be patient. This can take a few minutes.</h3>"))
    with video_output_area:
        clear_output(wait=True)

    # 1. Validate Source Image
    if source_image_path_global is None or not os.path.exists(source_image_path_global):
        with status_output:
            clear_output(wait=True)
            display(HTML("<h3 style='color: red;'>❌ Error: Please upload a character source image first.</h3>"))
        return

    # 2. Prepare Driving Audio
    current_driving_audio = None
    if audio_source_type.value == 'Upload Audio File':
        if driving_audio_path_global and os.path.exists(driving_audio_path_global):
            current_driving_audio = driving_audio_path_global
            with status_output: display(HTML(f"<p>Using uploaded audio: {os.path.basename(current_driving_audio)}</p>"))
        else:
            with status_output:
                clear_output(wait=True)
                display(HTML("<h3 style='color: red;'>❌ Error: Please upload an audio file or provide a valid YouTube URL.</h3>"))
            return
    elif audio_source_type.value == 'YouTube URL':
        yt_url = youtube_url_widget.value
        if not yt_url:
            with status_output:
                clear_output(wait=True)
                display(HTML("<h3 style='color: red;'>❌ Error: YouTube URL is empty.</h3>"))
            return
        try:
            with status_output: display(HTML(f"<p>Attempting to download audio from YouTube: {yt_url}</p>"))
            os.makedirs('inputs/yt_audio', exist_ok=True)
            audio_filename_base = f"inputs/yt_audio/youtube_audio_{int(time.time())}"
            ydl_opts = {
                'format': 'bestaudio/best',
                'outtmpl': audio_filename_base,
                'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'wav'}],
                'noplaylist': True,
                'quiet': False,
            }
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([yt_url])
            current_driving_audio = audio_filename_base + ".wav"
            if not os.path.exists(current_driving_audio):
                 potential_files = glob.glob(audio_filename_base + ".*wav") # More specific
                 if potential_files: current_driving_audio = potential_files[0]
                 else: # Try other common audio extensions if wav not found
                    potential_files = glob.glob(audio_filename_base + ".*")
                    if potential_files: current_driving_audio = potential_files[0]


            if not current_driving_audio or not os.path.exists(current_driving_audio):
                 with status_output:
                    clear_output(wait=True)
                    display(HTML("<h3 style='color: red;'>❌ Error: Could not download or find audio from YouTube URL. Check the URL and logs.</h3>"))
                 return
            with status_output: display(HTML(f"<p>Successfully processed YouTube audio: {os.path.basename(current_driving_audio)}</p>"))
            with uploaded_audio_display: # Show the downloaded audio
                clear_output(wait=True)
                display(Audio(current_driving_audio))

        except Exception as e:
            with status_output:
                clear_output(wait=True)
                display(HTML(f"<h3 style='color: red;'>❌ Error downloading/processing YouTube audio: {e}</h3>"))
            return
    else:
        with status_output:
            clear_output(wait=True)
            display(HTML("<h3 style='color: red;'>❌ Error: Invalid audio source selected.</h3>"))
        return

    # 3. Prepare SadTalker Command
    sadtalker_path = 'SadTalker'
    output_video_dir = os.path.join(sadtalker_path, 'results')

    if os.path.exists(output_video_dir):
        shutil.rmtree(output_video_dir)
    os.makedirs(output_video_dir, exist_ok=True)

    abs_source_image_path = os.path.abspath(source_image_path_global)
    abs_driving_audio_path = os.path.abspath(current_driving_audio)

    enhancer_val = enhancer_widget.value if enhancer_widget.value != 'None' else ''
    still_mode_val = str(still_mode_widget.value)

    sadtalker_command = f"python {sadtalker_path}/inference.py \
        --driven_audio \"{abs_driving_audio_path}\" \
        --source_image \"{abs_source_image_path}\" \
        --result_dir \"{output_video_dir}\" \
        --enhancer \"{enhancer_val}\" \
        --still {still_mode_val} \
        --preprocess full \
        --face3dvis_root \"{sadtalker_path}/checkpoints/face3d_pretrain_models\"" # Ensure this path is correct

    with status_output: display(HTML(f"<p>⚙️ Running SadTalker command...</p><pre style='font-size:0.8em; background:#f0f0f0; padding:5px; border-radius:3px; white-space:pre-wrap;'>{sadtalker_command}</pre>"))

    # 4. Run SadTalker
    try:
        command_to_run = f"cd {sadtalker_path} && python inference.py \
            --driven_audio \"{os.path.relpath(abs_driving_audio_path, sadtalker_path)}\" \
            --source_image \"{os.path.relpath(abs_source_image_path, sadtalker_path)}\" \
            --result_dir results \
            --enhancer \"{enhancer_val}\" \
            --still {still_mode_val} \
            --preprocess full \
            --face3dvis_root checkpoints/face3d_pretrain_models"
        # Using os.system for simplicity in Colab, but subprocess is more robust for complex cases
        # print(f"Executing: {command_to_run}") # For debugging
        os.system(command_to_run)
    except Exception as e:
        with status_output:
            clear_output(wait=True)
            display(HTML(f"<h3 style='color: red;'>❌ An error occurred during SadTalker execution: {e}</h3>"))
        return

    # 5. Display Result
    with status_output:
        clear_output(wait=True)
        display(HTML("<h3 style='color: green;'>✅ Animation generation complete!</h3>"))

    result_videos = sorted([os.path.join(output_video_dir, f) for f in os.listdir(output_video_dir) if f.endswith('.mp4')])

    if result_videos:
        latest_video_relative_to_sadtalker = result_videos[-1] # This path is like 'SadTalker/results/video.mp4'
        latest_video_for_display = latest_video_relative_to_sadtalker # Path is fine for FileLink if SadTalker is in /content/

        with video_output_area:
            clear_output(wait=True)
            display(HTML(f"<h4>Generated Video:</h4>"))
            # For HTML video tag, it needs to be accessible.
            # Copying to a known location outside SadTalker for simplicity, or use absolute path if known.
            # Since SadTalker is cloned into /content/SadTalker, the path should work directly.
            display(HTML(f"""
            <video width="80%" controls autoplay loop>
                  <source src="{latest_video_for_display}" type="video/mp4">
                  Your browser does not support the video tag.
            </video>
            """))
            # Provide download link by copying to the root content dir
            unique_output_filename = f"animated_video_{int(time.time())}.mp4"
            shutil.copy(latest_video_for_display, f"/content/{unique_output_filename}")
            display(FileLink(f"/content/{unique_output_filename}", result_html_prefix="<b style='font-size:1.1em;'>Download Video: </b>"))
    else:
        with video_output_area:
            clear_output(wait=True)
            display(HTML("<p style='color:red;'>😭 No video was found in the SadTalker results directory. Check logs above for errors.</p>"))


generate_button.on_click(on_generate_button_clicked)

# --- UI Layout ---
uploader_box = VBox([ # VBox is now defined
    source_image_uploader,
    uploaded_image_display
])

audio_box = VBox([ # VBox is now defined
    audio_source_type,
    youtube_url_widget,
    audio_file_uploader,
    uploaded_audio_display
])

settings_box = VBox([ # VBox is now defined
    enhancer_widget,
    still_mode_widget
])

# Main display
display(HTML("<h2>✨ Create Your Talking Character Animation ✨</h2>"))
display(HTML("<p>Follow the steps below to bring your character to life!</p><hr>"))
display(HBox([uploader_box, audio_box], layout=Layout(justify_content='space-around'))) # HBox is now defined
display(HTML("<hr>"))
display(settings_box)
display(generate_button)
display(HTML("<hr><h3>📢 Status & Logs:</h3>"))
display(status_output)
display(HTML("<hr><h3>🎬 Output Video:</h3>"))
display(video_output_area)

# Initial UI messages
with uploaded_image_display: display(HTML("<p style='color:grey;'>Preview your uploaded image here.</p>"))
with uploaded_audio_display: display(HTML("<p style='color:grey;'>Audio information will appear here.</p>"))
with status_output: display(HTML("<p style='color:grey;'>Status messages will appear here once you start.</p>"))
with video_output_area: display(HTML("<p style='color:grey;'>Your generated video will appear here.</p>"))

Button(button_style='success', description='🎬 Generate Animation!', icon='magic', layout=Layout(margin='20px 0…

Output()

Output()